In [1]:
# https://github.com/markwk/qs_ledger/blob/master/google_sheets/google_sheet_get_data_example.ipynb

### Make a more in depth tutorial on how to connect to your google sheets, 

* https://developers.google.com/sheets/api/quickstart/python
* https://developers.google.com/workspace/guides/create-credentials

Remember, desktop app, not web

In [2]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from datetime import date, datetime as dt, timedelta as td
import pandas as pd

In [3]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

### The Sheet ID and target range (i.e. sheet name) of a sample spreadsheet.
* Copy Sheet ID from URL of Targetted Sheet
* Add ID to Configuration below as well as sheet name and/or range.

In [4]:
# Configure to Your Specific Sheet 

# SAMPLE_SPREADSHEET_ID = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
# SAMPLE_RANGE_NAME = 'Class Data!A2:E'

SPREADSHEET_ID = "11JZUA0diTt4dlKDD_uSVLKkCT4E-QamC9I-weEzqsu6s"
RANGE_NAME = "Form Responses 1"

In [6]:
# https://stackoverflow.com/questions/11485271/google-oauth-2-authorization-error-redirect-uri-mismatch
# Authentication
creds = None

# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.

if os.path.exists('data/token.pickle'):
    with open('data/token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('data/credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# build service
service = build('sheets', 'v4', credentials=creds)

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [26]:
# Get Data from Google Sheet and Return a List
def get_gsheet_data(SPREADSHEET_ID, RANGE_NAME):
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])
    return values

In [27]:
# get data
data_list = get_gsheet_data(SPREADSHEET_ID, RANGE_NAME)
# type(data) #list

In [28]:
# create data and frame and use first row as header
data = pd.DataFrame(data_list)
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

In [29]:
data

,Group,Activity type,Duration,From,To,Comment
1,,Sleeping,7:10:21,2021-12-31 22:33:38,2022-01-01 5:43:59,None
2,,Grooming,0:02:21,2021-12-31 22:30:00,2021-12-31 22:32:21,None
3,,Toilet,0:01:02,2021-12-31 22:28:58,2021-12-31 22:30:00,None
4,,Eating,0:01:29,2021-12-31 18:32:06,2021-12-31 18:33:35,None
5,Computer,Internet,3:57:23,2021-12-31 18:31:31,2021-12-31 22:28:54,None
...,...,...,...,...,...,...
9473,Intimate,Sex,0:49:01,0.01,None,None
9474,,Shopping,18:17:36,0.19,None,None
9475,,Errands and Appointments,12:14:08,0.13,None,None
9476,,Phone,3:11:40,0.03,None,None


In [35]:
# export to csv
data.to_csv("data/notes.csv", index=False)